In [4]:
!pip install --quiet openai langchain-openai gradio requests sentence-transformers faiss-cpu

In [5]:
import os
import json
import requests
from dataclasses import dataclass, asdict
from typing import List, Dict, Any

import gradio as gr
from sentence_transformers import SentenceTransformer

In [6]:


# 1. SETUP & CSS
# ==========================================
# We define the Vintage Theme CSS here to ensure the "Booknerd" ambience.
VINTAGE_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Crimson+Text:ital,wght@0,400;0,600;1,400&family=Playfair+Display:wght@700&display=swap');

body, .gradio-container {
    background-color: #2b2118 !important; /* Dark Leather Brown */
    color: #e3d5c6 !important; /* Old Paper */
    font-family: 'Crimson Text', serif !important;
}

/* Titles */
.vintage-title {
    font-family: 'Playfair Display', serif;
    font-size: 3em;
    text-align: center;
    color: #d4af37; /* Gold Leaf */
    margin-bottom: 0.2em;
    text-shadow: 2px 2px 4px #000;
}
.vintage-subtitle {
    text-align: center;
    font-size: 1.2em;
    font-style: italic;
    opacity: 0.8;
    margin-bottom: 2em;
    border-bottom: 1px solid #d4af37;
    padding-bottom: 1em;
}

/* Inputs (Checkboxes, Sliders) */
label span {
    color: #e3d5c6 !important;
    font-size: 1.1em !important;
}
.block.gradio-checkboxgroup, .block.gradio-slider, .block.gradio-dropdown {
    background: #1f1812 !important;
    border: 1px solid #5c4033 !important;
    border-radius: 8px;
    padding: 15px;
}

/* Buttons */
.vintage-button {
    background-color: #5c4033 !important; /* Dark Wood */
    color: #fff !important;
    border: 1px solid #d4af37 !important;
    font-family: 'Playfair Display', serif !important;
    font-size: 1.2em !important;
    transition: all 0.3s ease;
}
.vintage-button:hover {
    background-color: #d4af37 !important;
    color: #1f1812 !important;
    box-shadow: 0 0 10px #d4af37;
}

/* Section Headers */
.vintage-section-title {
    font-family: 'Playfair Display', serif;
    font-size: 1.8em;
    margin-top: 30px;
    margin-bottom: 15px;
    color: #d4af37;
    border-left: 4px solid #d4af37;
    padding-left: 10px;
}

/* Scrollbar */
::-webkit-scrollbar {
    width: 10px;
}
::-webkit-scrollbar-track {
    background: #1f1812;
}
::-webkit-scrollbar-thumb {
    background: #5c4033;
}
"""

# 2. LOGIC & AGENTS
# ==========================================

# Initialize Embedding Model (keep silent)
try:
    EMBED_MODEL = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
except:
    pass # Handle gracefully if already loaded

class SimpleMemory:
    def __init__(self, path="booknerd_memory.json"):
        self.path = path
        if not os.path.exists(self.path):
            with open(self.path, "w") as f:
                json.dump({"favorites": [], "save_later": []}, f)
        self.load()

    def load(self):
        with open(self.path, "r") as f:
            self.data = json.load(f)

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self.data, f, indent=2)

    def add(self, key, entry):
        self.data.setdefault(key, [])
        # Prevent duplicates based on ID
        if not any(e["id"] == entry["id"] for e in self.data[key]):
            self.data[key].append(entry)
            self.save()

    def get(self, key):
        return self.data.get(key, [])

memory = SimpleMemory()

@dataclass
class UserProfile:
    genres: List[str]

class ProfileAnalyzerAgent:
    def analyze(self, genres_list):
        cleaned = [g.strip().lower() for g in genres_list] if genres_list else ["fiction"]
        return UserProfile(genres=cleaned)

class SearchDataAgent:
    # Google Books API Endpoint
    BASE_URL = "https://www.googleapis.com/books/v1/volumes"

    def search(self, subject, max_results=40):
        try:
            # Google Books uses 'subject:genre' filtering
            params = {
                "q": f"subject:{subject}",
                "maxResults": max_results,
                "printType": "books",
                "orderBy": "relevance"  # 'relevance' is better for finding definitive/classic editions
            }

            # Send Request
            r = requests.get(self.BASE_URL, params=params, timeout=10)
            if not r.ok:
                print(f"API Error: {r.status_code}")
                return []

            data = r.json()
            items = data.get("items", [])
            results = []

            for item in items:
                info = item.get("volumeInfo", {})

                # 1. Extract Basic Info
                title = info.get("title", "Untitled")
                authors = info.get("authors", ["Unknown"])
                categories = info.get("categories", []) # Google's version of subjects
                description = info.get("description", "No description available.")

                # 2. Extract Year (Handle "YYYY", "YYYY-MM-DD", etc.)
                pub_date = info.get("publishedDate", "")
                year = None
                if pub_date:
                    # Take the first 4 digits found
                    year_str = pub_date[:4]
                    if year_str.isdigit():
                        year = int(year_str)

                # 3. Extract Cover (Prefer larger, fall back to thumbnail)
                # Google images often come as http, which breaks in some secure apps. Force https.
                images = info.get("imageLinks", {})
                cover = images.get("thumbnail") or images.get("smallThumbnail")

                if cover:
                    cover = cover.replace("http://", "https://")
                else:
                    cover = "https://via.placeholder.com/300x450/2b2118/e3d5c6?text=No+Cover"

                # 4. Build Record
                results.append({
                    "id": item.get("id"), # Google's unique Volume ID
                    "title": title,
                    "authors": authors,
                    "subjects": categories,
                    "cover": cover,
                    "year": year,
                    "description": description # Added bonus field
                })
            return results

        except Exception as e:
            print(f"Search error: {e}")
            return []

class CriticAgent:
    def score(self, profile, books):
        results = []
        for b in books:
            score = 0
            subjects = [s.lower() for s in b.get("subjects", [])]
            # Simple scoring: match genres
            for g in profile.genres:
                if any(g in s for s in subjects):
                    score += 2

            # Bonus for having a valid year (data quality)
            if b.get("year"):
                score += 1

            results.append({"book": b, "score": score})

        # Sort by score descending
        return sorted(results, key=lambda x: x["score"], reverse=True)

class Orchestrator:
    def __init__(self, profile_agent, search_agent, critic_agent, memory):
        self.profile_agent = profile_agent
        self.search_agent = search_agent
        self.critic_agent = critic_agent
        self.memory = memory

    def recommend(self, genres, k=5, year_range=(1800, 2024)):
        profile = self.profile_agent.analyze(genres)
        all_books = []

        min_year, max_year = year_range

        for genre in profile.genres:
            found = self.search_agent.search(genre)
            all_books.extend(found)

        # 1. Deduplicate
        unique_books = {b["id"]: b for b in all_books}.values()

        # 2. Filter by Year
        filtered_books = []
        for b in unique_books:
            y = b.get("year")
            # If year is unknown, we include it cautiously, or exclude.
            # Let's exclude unknown years if strictly filtering,
            # but to be nice, we'll include them if within reasonable bounds logic.
            # Here: Only include if year is known and within range.
            if y and (min_year <= y <= max_year):
                filtered_books.append(b)

        # 3. Score
        scored = self.critic_agent.score(profile, filtered_books)
        top = scored[:k]

        results = []
        for s in top:
            b = s["book"]
            results.append({
                **b,
                "score": s["score"],
                "explanation": f"Matches themes in {', '.join(genres)}"
            })

        global LAST_RECOMMENDATIONS
        LAST_RECOMMENDATIONS = results
        return results

# Initialize Agents
profile_agent = ProfileAnalyzerAgent()
search_agent = SearchDataAgent()
critic_agent = CriticAgent()
orch = Orchestrator(profile_agent, search_agent, critic_agent, memory)

LAST_RECOMMENDATIONS = []

# 3. FORMATTING & UI HELPERS
# ==========================================

def format_cards(books):
    if not books:
        return "<p style='color:#d4af37; text-align:center;'>No volumes found in the archives for this era.</p>"

    html = "<div style='display: flex; flex-wrap: wrap; gap: 20px; justify-content: center;'>"

    for b in books:
        title = b.get("title")
        authors = ", ".join(b.get("authors", []))
        year = b.get("year") or "N/A"
        cover = b.get("cover")

        html += f"""
        <div style="
            width: 250px;
            background: #1f1812;
            border: 1px solid #5c4033;
            border-radius: 8px;
            padding: 15px;
            box-shadow: 5px 5px 15px rgba(0,0,0,0.5);
            display: flex; flex-direction: column;
            transition: transform 0.2s;
        ">
            <div style="height: 350px; overflow: hidden; border-radius: 4px; margin-bottom: 10px;">
                <img src="{cover}" style="width: 100%; height: 100%; object-fit: cover;">
            </div>
            <h3 style="margin: 5px 0; color: #d4af37; font-size: 1.1em; line-height: 1.2;">{title}</h3>
            <p style="font-size: 0.9em; opacity: 0.9; margin: 0;"><i>by {authors}</i></p>
            <p style="font-size: 0.85em; color: #8b5a2b; margin-top: 5px;">Published: {year}</p>
        </div>
        """
    html += "</div>"
    return html

def render_saved_list(key):
    items = memory.get(key)
    if not items:
        return "<p style='color:#777; font-style:italic;'>The shelf is empty.</p>"

    html = "<div style='display:flex; flex-wrap:wrap; gap:15px;'>"
    for b in items:
        html += f"""
        <div style="width:180px; padding:10px; background:#221e1a; border:1px solid #3e2f26; border-radius:6px;">
            <img src="{b['cover']}" style="width:100%; height:200px; object-fit:cover; border-radius:4px;">
            <h4 style="font-size:14px; color:#d4af37; margin:5px 0;">{b['title']}</h4>
        </div>
        """
    html += "</div>"
    return html

# 4. GRADIO INTERFACE FUNCTIONS
# ==========================================

GENRE_OPTIONS = [
    "Fiction", "Mystery", "Thriller", "Romance", "Fantasy",
    "Science Fiction", "Historical Fiction", "Horror", "Classics",
    "Philosophy", "Psychology", "History", "Poetry"
]

def recommend_ui(genres, quantity, min_year, max_year):
    if not genres:
        gr.Info("Please select at least one genre to explore.")
        return None, gr.update(choices=[], value=None)

    books = orch.recommend(genres, quantity, year_range=(min_year, max_year))

    if not books:
        gr.Info("No books found matching these specific criteria.")
        return format_cards([]), gr.update(choices=[], value=None)

    # Update dropdown options
    labels = [f"{b['title']} ({b.get('year', 'N/A')})" for b in books]
    return format_cards(books), gr.update(choices=labels, value=labels[0] if labels else None)

def add_fav_ui(selected_label):
    if not selected_label:
        gr.Info("No book selected.")
        return
    # Find book object
    for b in LAST_RECOMMENDATIONS:
        label = f"{b['title']} ({b.get('year', 'N/A')})"
        if label == selected_label:
            memory.add("favorites", b)
            gr.Info(f"Saved to Favorites: {b['title']}")
            return
    gr.Info("Error: Book data not found.")

def add_save_ui(selected_label):
    if not selected_label:
        gr.Info("No book selected.")
        return
    for b in LAST_RECOMMENDATIONS:
        label = f"{b['title']} ({b.get('year', 'N/A')})"
        if label == selected_label:
            memory.add("save_later", b)
            gr.Info(f"Saved for Later: {b['title']}")
            return
    gr.Info("Error: Book data not found.")

# 5. BUILD THE UI
# ==========================================

with gr.Blocks(css=VINTAGE_CSS, theme=gr.themes.Default(primary_hue="yellow")) as demo:

    gr.HTML("<div class='vintage-title'>🕯️ The BookNerd's Sanctuary</div>")
    gr.HTML("<div class='vintage-subtitle'>Curated literary recommendations from the archives.</div>")

    with gr.Row():
        with gr.Column(scale=1):
            # CONTROLS
            gr.Markdown("### 1. Define Your Tastes")
            genres = gr.CheckboxGroup(GENRE_OPTIONS, label="Select Genres")

            gr.Markdown("### 2. Time Period")
            min_year_slider = gr.Slider(
                minimum=1800, maximum=2025, value=1900, step=1, label="Min Publication Year"
            )
            max_year_slider = gr.Slider(
                minimum=1800, maximum=2025, value=2024, step=1, label="Max Publication Year"
            )

            qty = gr.Slider(1, 10, value=4, step=1, label="Stack Size")

            btn = gr.Button("🔍 Consult the Archives", elem_classes="vintage-button")

            gr.Markdown("---")

            # ACTIONS
            gr.Markdown("### 3. Actions")
            # We keep the dropdown so the buttons know WHAT to save, but simplify it.
            book_selector = gr.Dropdown([], label="Select Book from Results")

            with gr.Row():
                fav_btn = gr.Button("❤️ Add to Favorites", elem_classes="vintage-button")
                save_btn = gr.Button("🔖 Save for Later", elem_classes="vintage-button")

        with gr.Column(scale=2):
            # DISPLAY
            gr.Markdown("### Recommended Volumes")
            out_display = gr.HTML()

            gr.Markdown("---")

            with gr.Tabs():
                with gr.TabItem("⭐ Favorites"):
                    fav_view = gr.HTML(render_saved_list("favorites"))
                    refresh_fav = gr.Button("Refresh Shelf")
                with gr.TabItem("📚 To Read Later"):
                    save_view = gr.HTML(render_saved_list("save_later"))
                    refresh_save = gr.Button("Refresh Shelf")

    # WIRING
    btn.click(recommend_ui, [genres, qty, min_year_slider, max_year_slider], [out_display, book_selector])

    fav_btn.click(add_fav_ui, book_selector, None)
    save_btn.click(add_save_ui, book_selector, None)

    # Refresh Logic for shelves
    refresh_fav.click(lambda: render_saved_list("favorites"), None, fav_view)
    refresh_save.click(lambda: render_saved_list("save_later"), None, save_view)

demo.launch(debug=True)

/tmp/ipython-input-2201629454.py:377: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=VINTAGE_CSS, theme=gr.themes.Default(primary_hue="yellow")) as demo:
/tmp/ipython-input-2201629454.py:377: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=VINTAGE_CSS, theme=gr.themes.Default(primary_hue="yellow")) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d84649835995293abd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d84649835995293abd.gradio.live
